# SIPALAGA EXTRACTOR

##### Oleh: Tarida Priskila Hasian Simanjuntak



#### Deskripsi Program:
Program ini digunakan untuk mengolah dan mengcompile data SIPALAGA di Provinsi Riau ke dalam satu buah tabel.
Program masih menerima input yang disuplai oleh user secara manual dengan cara mengunduh file .pdf di website
SIPALAGA. Harapannya, program ini dapat mengunduh data secara otomatis dari website SIPALAGA. Dengan demikian, 
intevensi user dapat diminimalisir. 

Di akhir program, data berupa besar GWL, SM, dan Curah Hujan rataan di seluruh Provinsi Riau pada suatu tanggal tertentu akan tersimpan ke dalam sebuah file .xlsx.

Program ini terdiri atas 4 stage:
1. Pengolahan data SIPALAGA dengan status "Rawan" ke dalam 1 Data Frame
2. Pengolahan data SIPALAGA dengan status selain "Rawan" ke dalam 1 Data Frame
3. Penggabungan 2 Data Frame menjadi 1 Data Frame akhir
4. Eksport Data Frame ke dalam bentuk .xlsx

Sebelum menjalankan program, user harus melakukan preparasi data:
1. Mendownload semua data pdf SIPALAGA
2. Mengelompokkan pdf ke dalam folder-folder sesuai dengan status TMAT (Rawan & selain Rawan)
3. Menyiapkan folder/directory untuk menyimpan file akhir .xlsx yang akan menjadi output akhir dari program ini
4. Memastikan tidak ada file .pdf yang salah folder/directory

Penjelasan detil akan diberikan pada masing-masing stage.

## STAGE 1: PENGOLAHAN DATA SIPALAGA DENGAN STATUS "RAWAN"

Pada stage ini, user harus memastikan path yang tertera pada code di bawah ini sudah sesuai dengan path dari folder tempat penympanan pdf SIPALAGA dengan status "Rawan". Selain alamat path, tidak ada variabel yang perlu diubah pada stage ini. 

##### Path dengan format: r"path\nama file"

catatan: "r" jangan sampai ketinggalan", nama file tanpa .pdf

In [1]:
import pandas as pd
import pdfplumber
import glob
import os
import datetime
import time
import numpy as np

Status = []
dat_list = []
gwl_list = []
sm_list = []
ch_list = []
status_list = []
stasiun = []
kecamatan = []
kabupaten = []

Status1 = []
dat_list1 = []
gwl_list1 = []
sm_list1 = []
ch_list1 = []
status_list1 = []
stasiun1 = []
kecamatan1 = []
kabupaten1 = []

pdf_files = glob.glob(os.path.join(r"C:\Users\Tarida Simanjuntak\SIPALAGA\RAWAN\pdf_brg", "*.pdf")) #PATH ADA DI SINI

for p in pdf_files:
    with pdfplumber.open(p) as pdf:
        first_page = pdf.pages[0]
        p0 = first_page.extract_text()
        lst_splt = p0.split("\n")
        del lst_splt[:11]
        del lst_splt[-1]
        lst_splt = [k.replace(" ",',') for k in lst_splt]
        lst_splt = [k.split(',') for k in lst_splt]
        df = pd.DataFrame(lst_splt)
        name = ["Jam", "GWL_Mean", "GWL_Min", "GWL_Max", "SM_Mean", "SM Min", "SM Max", "CH Mean", "CH Min", "CH Max","Status"]
        df.columns = name
        #df[["Jam"]] = pd.to_datetime(df["Jam"],format='%H:%M:%S').dt.time
        df[["Temp"]] = pd.to_datetime(df["Jam"])
        df[["Jam"]] = df["Temp"].dt.time
        df[["GWL_Mean", "GWL_Min", "GWL_Max", "SM_Mean", "SM Min", "SM Max", "CH Mean", "CH Min", "CH Max"]] = df[["GWL_Mean", "GWL_Min", "GWL_Max", "SM_Mean", "SM Min", "SM Max", "CH Mean", "CH Min", "CH Max"]].apply(pd.to_numeric)
        for i in range(len(df.index)):
            dat1a = df.loc[i].at["Jam"]
            dat1b = df.iloc[i-1].at["Jam"]
            dat2 = datetime.time(7,0,0)   
            dat3 = datetime.time(9,58,20)
            if (dat1a >= dat2)and (dat1a <= dat3):
                dat = dat1a
                gwl_dat = df.loc[i].at["GWL_Mean"]
                sm_dat = df.loc[i].at["SM_Mean"]
                ch_dat = df.loc[i].at["CH Mean"] 
                status_dat = df.loc[i].at["Status"]
                break
            elif (dat1a >= dat2) and (dat1a >= dat3):
                dat = dat1b
                gwl_dat = df.iloc[i-1].at["GWL_Mean"]
                sm_dat = df.iloc[i-1].at["SM_Mean"]
                ch_dat = df.iloc[i-1].at["CH Mean"] 
                status_dat = df.iloc[i-1].at["Status"]
            else:
                dat = dat1a
                gwl_dat = df.iloc[1].at["GWL_Mean"]
                sm_dat = df.iloc[1].at["SM_Mean"]
                ch_dat = df.iloc[1].at["CH Mean"] 
                status_dat = df.iloc[1].at["Status"]
    
                
    dat_list.append(dat)
    gwl_list.append(gwl_dat)
    sm_list.append(sm_dat)
    ch_list.append(ch_dat)
    status_list.append(status_dat)


for p in pdf_files:
        with pdfplumber.open(p) as pdf:
            first_page = pdf.pages[0]
            p0 = first_page.extract_text()
            lst_splt = p0.split("\n")
            del lst_splt[0]
            del lst_splt[5:]
            info = [k.split(" : ") for k in lst_splt]
            sta = info[0][1]
            kec = info[1][1]
            kab=info[3][1]
            stasiun.append(sta)
            kecamatan.append(kec)
            kabupaten.append(kab)    


for p in pdf_files:
        with pdfplumber.open(p) as pdf:
            first_page = pdf.pages[0]
            p0 = first_page.extract_text()
            lst_splt = p0.split("\n")
            del lst_splt[0]
            del lst_splt[5:]
            info = [k.split(" : ") for k in lst_splt]
            sta = info[0][1]
            kec = info[1][1]
            kab=info[3][1]
            stasiun.append(sta)
            kecamatan.append(kec)
            kabupaten.append(kab)    

DataSipalagaR_df = pd.DataFrame(list(zip(stasiun, kecamatan, kabupaten, dat_list, gwl_list,sm_list,ch_list,status_list)),
                  columns =['Stasiun BRG', 'Kecamatan', 'Kabupaten','Jam', 'GWL Rataan','SM Rataan','CH Rataan','Status'])

DataSipalagaR_df


,Stasiun BRG,Kecamatan,Kabupaten,Jam,GWL Rataan,SM Rataan,CH Rataan,Status
0,BRG_140701_01,SEI SEGAJAH MAKMUR 1,ROKAN HILIR,07:00:08,-0.89,43.55,0,RAWAN
1,BRG_140312_01,SUMBER JAYA,BENGKALIS,07:00:01,-1.51,19.00,0,RAWAN
2,BRG_140312_02,SADAR JAYA,BENGKALIS,07:00:00,-0.47,85.78,0,RAWAN
3,BRG_140801_01,KAMPUNG REMPAK,SIAK,08:30:08,-1.72,67.90,0,RAWAN
4,BRG_140806_01,DAYUN,SIAK,07:00:00,-0.45,78.58,0,RAWAN
5,BRG_140802_02,PENYENGAT 2,SIAK,07:00:06,-1.22,15.97,0,RAWAN
6,BRG_140802_01,PENYENGAT 1,SIAK,07:00:01,-0.68,48.30,0,RAWAN
7,BRG_141006_01,SEMUKUT,KEPULAUAN MERANTI,07:00:00,-0.48,58.15,0,RAWAN
8,BRG_141009_01,KEDABURAPAT,KEPULAUAN MERANTI,07:00:00,-0.64,86.67,0,RAWAN
9,BRG_141003_01,TANJUNG BAKAU,KEPULAUAN MERANTI,07:00:00,-0.68,77.77,0,RAWAN


## STAGE 2: PENGOLAHAN DATA SIPALAGA DENGAN STATUS SELAIN "RAWAN"

Pada stage ini, user harus memastikan path yang tertera pada code di bawah ini sudah sesuai dengan path dari folder tempat penympanan pdf SIPALAGA dengan status SELAIN "Rawan". Selain alamat path, tidak ada variabel yang perlu diubah pada stage ini. 

##### Path dengan format: r"path\nama file"

catatan: "r" jangan sampai ketinggalan", nama file tanpa .pdf



In [2]:

pdf_file = glob.glob(os.path.join(r"C:\Users\Tarida Simanjuntak\SIPALAGA\KURANG_RAWAN\brgkr_pdf", "*.pdf")) #PATH ADA DI SINI

for p in pdf_file:
    with pdfplumber.open(p) as pdf:
        first_page1 = pdf.pages[0]
        p01 = first_page1.extract_text()
        lst_splt1 = p01.split("\n")
        del lst_splt1[:11]
        del lst_splt1[-1]
        lst_splt1 = [k.replace(" ",',') for k in lst_splt1]
        lst_splt1 = [k.split(',') for k in lst_splt1]
        df_new = pd.DataFrame(lst_splt1)
        name1 = ["Jam", "GWL_Mean", "GWL_Min", "GWL_Max", "SM_Mean", "SM Min", "SM Max", "CH Mean", "CH Min", "CH Max"]
        df_new.columns = name1
        df_new = df_new[df_new.GWL_Min.notnull()]
        ind1 = [x for x in range (len(df_new.index))]
        df_new.index = ind1
        df_new[["GWL_Mean", "GWL_Min", "GWL_Max", "SM_Mean", "SM Min", "SM Max", "CH Mean", "CH Min", "CH Max"]] = df_new[["GWL_Mean", "GWL_Min", "GWL_Max", "SM_Mean", "SM Min", "SM Max", "CH Mean", "CH Min", "CH Max"]].apply(pd.to_numeric)
        df_new["Status"] = np.nan
        for x in ind1:
            gwl = df_new.iloc[x].at["GWL_Mean"]
            if  (gwl <0):
                if (gwl < -0.4):
                    df_new.Status = df_new.Status.fillna('RAWAN')
                elif (gwl >= -0.4):
                     df_new.Status = df_new.Status.fillna('KURANG RAWAN')
            elif gwl > 0:
                df_new.Status = df_new.Status.fillna('TIDAK RAWAN')
            elif gwl == 0:
                df_new.Status = df_new.Status.fillna('KOSONG')
            else:
                df_new.Status = df_new.Status.fillna('KOSONG')
        df_new[["Temp"]] = pd.to_datetime(df_new["Jam"])
        df_new[["Jam"]] = df_new["Temp"].dt.time
        del df_new["Temp"]
        for i in range(len(df_new.index)):
            dat1a1 = df_new.loc[i].at["Jam"]
            dat1b1 = df_new.iloc[i-1].at["Jam"]
            dat21 = datetime.time(7,0,0)   
            dat31 = datetime.time(9,58,20)
            if (dat1a1 >= dat21)and (dat1a1 <= dat31):
                dat = dat1a1
                gwl_dat1 = df_new.loc[i].at["GWL_Mean"]
                sm_dat1 = df_new.loc[i].at["SM_Mean"]
                ch_dat1 = df_new.loc[i].at["CH Mean"] 
                status_dat1 = df_new.loc[i].at["Status"]
                break
            elif (dat1a1 >= dat21) and (dat1a1 >= dat31):
                dat = dat1b1
                gwl_dat1 = df_new.iloc[i-1].at["GWL_Mean"]
                sm_dat1 = df_new.iloc[i-1].at["SM_Mean"]
                ch_dat1 = df_new.iloc[i-1].at["CH Mean"] 
                status_dat1 = df_new.iloc[i-1].at["Status"]
            else:
                dat = dat1a1
                gwl_dat1 = df_new.iloc[1].at["GWL_Mean"]
                sm_dat1 = df_new.iloc[1].at["SM_Mean"]
                ch_dat1 = df_new.iloc[1].at["CH Mean"] 
                status_dat1 = df_new.iloc[1].at["Status"]
    dat_list1.append(dat)
    gwl_list1.append(gwl_dat1)
    sm_list1.append(sm_dat1)
    ch_list1.append(ch_dat1)
    status_list1.append(status_dat1)
    
for p in pdf_file:
    with pdfplumber.open(p) as pdf:
        first_page1 = pdf.pages[0]
        p01 = first_page1.extract_text()
        lst_splt1 = p01.split("\n")
        del lst_splt1[0]
        del lst_splt1[5:]
        info = [k.split(" : ") for k in lst_splt1]
        sta1 = info[0][1]
        kec1 = info[1][1]
        kab1=info[3][1]
        stasiun1.append(sta1)
        kecamatan1.append(kec1)
        kabupaten1.append(kab1)   

DataSipalagaKR_df = pd.DataFrame(list(zip(stasiun1, kecamatan1, kabupaten1, dat_list1, gwl_list1,sm_list1,ch_list1,status_list1)),
                   columns =['Stasiun BRG', 'Kecamatan', 'Kabupaten','Jam', 'GWL Rataan','SM Rataan','CH Rataan','Status'])

DataSipalagaKR_df

       

,Stasiun BRG,Kecamatan,Kabupaten,Jam,GWL Rataan,SM Rataan,CH Rataan,Status
0,BRG_140701_02,TELUK PIYAI,ROKAN HILIR,07:00:04,1.19,66.53,0.0,TIDAK RAWAN
1,BRG_140402_01,BAGAN JAYA,INDRAGIRI HILIR,07:00:04,-0.24,16.40,0.0,KURANG RAWAN
2,BRG_140103_01,RIMBA PANJANG,KAMPAR,07:00:00,-0.38,78.01,0.0,KURANG RAWAN
3,BRG7,RIMBA PANJANG,KAMPAR,07:00:00,-0.13,55.72,0.0,KURANG RAWAN
4,BRG_140702_02,LABUHAN TANGGA BESAR 2,ROKAN HILIR,07:00:05,-0.25,22.40,0.0,KURANG RAWAN
5,BRG_140302_02,MUNTAI,BENGKALIS,07:00:00,-0.07,14.23,0.0,KURANG RAWAN
6,BRG_140302_02,MUNTAI,BENGKALIS,07:00:00,-0.15,12.83,0.0,KURANG RAWAN
7,BRG_140809_01,BUATAN I,SIAK,07:00:00,-0.36,0.00,0.0,KURANG RAWAN
8,BRG_140508_01,MERBAU 1 (EX DEMPLOT),PELALAWAN,07:00:03,-0.30,86.20,0.0,KURANG RAWAN
9,BRG_140508_02,MERBAU 2,PELALAWAN,07:00:04,-0.10,72.40,0.0,KURANG RAWAN


## STAGE 3: PENGGABUNGAN DATAFRAME PADA STAGE 1 DAN 2 MENJADI SATU DATAFRAME

Pada stage ini, tidak perlu ada variabel yang diubah pada stage ini. Stage ini akan menampilkan Dataframe akhir.

In [3]:
df_sipalaga_merged1 = pd.concat([DataSipalagaR_df, DataSipalagaKR_df], ignore_index=True)
df_sipalaga_merged1

,Stasiun BRG,Kecamatan,Kabupaten,Jam,GWL Rataan,SM Rataan,CH Rataan,Status
0,BRG_140701_01,SEI SEGAJAH MAKMUR 1,ROKAN HILIR,07:00:08,-0.89,43.55,0.0,RAWAN
1,BRG_140312_01,SUMBER JAYA,BENGKALIS,07:00:01,-1.51,19.00,0.0,RAWAN
2,BRG_140312_02,SADAR JAYA,BENGKALIS,07:00:00,-0.47,85.78,0.0,RAWAN
3,BRG_140801_01,KAMPUNG REMPAK,SIAK,08:30:08,-1.72,67.90,0.0,RAWAN
4,BRG_140806_01,DAYUN,SIAK,07:00:00,-0.45,78.58,0.0,RAWAN
5,BRG_140802_02,PENYENGAT 2,SIAK,07:00:06,-1.22,15.97,0.0,RAWAN
6,BRG_140802_01,PENYENGAT 1,SIAK,07:00:01,-0.68,48.30,0.0,RAWAN
7,BRG_141006_01,SEMUKUT,KEPULAUAN MERANTI,07:00:00,-0.48,58.15,0.0,RAWAN
8,BRG_141009_01,KEDABURAPAT,KEPULAUAN MERANTI,07:00:00,-0.64,86.67,0.0,RAWAN
9,BRG_141003_01,TANJUNG BAKAU,KEPULAUAN MERANTI,07:00:00,-0.68,77.77,0.0,RAWAN


## STAGE 4: PENYIMPANAN DATAFRAME KE DALAM FILE .xlsx

Pada stage ini, user harus memastikan telah memasukkan alamat path yang sesuai dengan alamat path tempat folder/directory yang akan digunakan untuk menyimpan file .xlsx.


#### format: r'alamat_path\nama_file.xlsx'

Jangan lupa 'r'

In [5]:
df_sipalaga_merged1.to_excel(r'C:\Users\Tarida Simanjuntak\SIPALAGA\Sipalaga_Holder\Sipalaga_20210715.xlsx', index = False) #PATH ADA DI SINI